In [1]:
import numpy as np

In [2]:
from sklearn.metrics import roc_auc_score

In [3]:
import sklearn.neighbors as nb
from sklearn.preprocessing import StandardScaler

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
import category_encoders

In [5]:
from sklearn.decomposition import PCA

In [6]:
from sklearn.model_selection import GridSearchCV

In [7]:
import pandas as pd

In [8]:
kaggle = pd.read_csv('trocafone_kaggle_test.csv')

In [9]:
labels = pd.read_csv('labels_training_set.csv')

In [10]:
processed = pd.read_csv('features_13.csv')

In [11]:
processed = processed.set_index('person').fillna(0)

## Mean Encoding

In [12]:
training_features = labels.set_index('person').join(processed)
predict_features = kaggle.set_index('person').join(processed)
training_labels = training_features['label']
training_features = training_features.drop('label',axis=1)

In [13]:
tg = category_encoders.TargetEncoder()
encoded_training = tg.fit_transform(np.array(training_features),np.array(training_labels))
encoded_predict = tg.transform(np.array(predict_features))

## Aplico una reduccion de dimensiones, de 118 a 90

In [15]:
pca = PCA(n_components = 90)

In [16]:
scaler = StandardScaler()
train_data_pca = pca.fit_transform(scaler.fit_transform(encoded_training.fillna(0)))
predict_data_pca = pca.fit_transform(scaler.fit_transform(encoded_predict.fillna(0)))

C:\Users\Usuario\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:617: DataConversionWarning: Data with input dtype bool, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\Usuario\Anaconda3\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype bool, int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
C:\Users\Usuario\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:617: DataConversionWarning: Data with input dtype bool, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\Usuario\Anaconda3\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype bool, int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


## Normalizo

In [18]:
train_features_norm = StandardScaler().fit_transform(train_data_pca)
predict_features_norm = StandardScaler().fit_transform(predict_data_pca)

## Clasificador KNN

In [18]:
clasKNN= nb.KNeighborsClassifier(n_neighbors=10000, weights='uniform', algorithm='auto', leaf_size=30, p=3, metric='minkowski', metric_params=None, n_jobs=None)

In [26]:
xtr,xte,ytr,yte=train_test_split(train_features_norm,training_labels)

In [27]:
clasKNN.fit(xtr,ytr)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=10000, p=3,
           weights='uniform')

In [28]:
roc_auc_score(yte,clasKNN.predict_proba(xte)[:,1])

0.7741699803175784

# GridSearch de parametros de KNN

In [19]:
params={
    'n_neighbors':[50,100,1000,2500,1000,5000],
    'p':[1,2,3,4,5,6,7],
    'metric': ["euclidean","chebyshev","minkowski"]
}
grid = GridSearchCV(nb.KNeighborsClassifier(),params,scoring='roc_auc',cv=2,verbose=0)

In [20]:
xtr,xte,ytr,yte=train_test_split(train_features_norm,training_labels,train_size=0.6,stratify=np.array(training_labels))

C:\Users\Usuario\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2069: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [21]:
grid.fit(xtr,ytr)

GridSearchCV(cv=2, error_score='raise-deprecating',
       estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform'),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'n_neighbors': [50, 100, 1000, 2500, 1000, 5000], 'p': [1, 2, 3, 4, 5, 6, 7], 'metric': ['euclidean', 'chebyshev', 'minkowski']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=0)

In [23]:
grid.score(xte,yte)

0.8158470467112802

In [24]:
grid.best_params_

{'metric': 'euclidean', 'n_neighbors': 1000, 'p': 1}

In [25]:
knn_clasf=grid.best_estimator_

In [27]:
roc_auc_score(yte,knn_clasf.predict_proba(xte)[:,1])

0.8158470467112802

## Pruebo otro split

In [39]:
xtr,xte,ytr,yte=train_test_split(train_features_norm,training_labels,train_size=0.6,stratify=np.array(training_labels))

C:\Users\Usuario\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2069: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [43]:
knn_clasf.fit(train_features_norm,training_labels)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='euclidean',
           metric_params=None, n_jobs=None, n_neighbors=1000, p=1,
           weights='uniform')

In [45]:
pred = knn_clasf.predict_proba(predict_features_norm)[:,1]

In [80]:
pred

array([0.033, 0.015, 0.015, ..., 0.027, 0.025, 0.031])

In [81]:
submit = kaggle.set_index('person')

In [82]:
submit['label'] = pred

In [83]:
submitXG = pd.read_csv("submitXG.csv").set_index("person")

In [84]:
submit.columns=["labelKNN"]

In [85]:
submit["labelXG"] = submitXG["label"]

In [86]:
submit["label"] = submit["labelKNN"]*0.3 + submit["labelXG"]*0.7

In [87]:
submit = submit.drop(["labelKNN","labelXG"], axis=1)

In [88]:
submit.to_csv("submit.csv")